# Parameter sharing (p.40)

In [1]:
import tensorflow as tf

with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var1 = tf.get_variable("var", [1])
        scp.reuse_variables()
        var2 = tf.get_variable("var", [1])
        
    with tf.variable_scope("bar", reuse=True):
        var3 = tf.get_variable("var", [1])
        
print("var1: {}".format(var1.name))
print("var2: {}".format(var2.name))
print("var3: {}".format(var3.name))

var1: foo/bar/var:0
var2: foo/bar/var:0
var3: foo/bar/var:0


# Wrappers (p.45)

In [3]:
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W", [ksize, ksize, bottom_shape, num_filter], initializer=he_init)
        b = tf.get_variable("b", [num_filter], initializer=tf.constant_initializer(0))
        
        x = tf.nn.conv2d(bottom, W, strides=[1, stride, stride, 1], padding=padding)
        x = tf.nn.relu(tf.nn.bias_add(x, b))
        
    return x

def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding=padding)
    return pool


def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims], initializer=he_init)
        b = tf.get_variable("b", [num_dims], initializer=tf.constant_initializer(0))
        
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
        
        return relu

# All Together (p.50)

In [4]:
keep_prob = tf.placeholder(tf.float32, None)

def conv_net(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope="out")
    return out

# Session config (p.52)

In [5]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)

# Layers (p.56)

In [6]:
import tensorflow as tf

input = ...
with tf.name_scope('conv1_1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32, stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(input, kernel, [1, 1, 1, 1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope)

TypeError: Expected binary or unicode string, got Ellipsis

In [7]:
import tensorflow.contrib.slim as slim

net = ...
net = slim.conv2d(net, 256, [3, 3], scope='conv3_1')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_2')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_3')
net = slim.max_pool2d(net, [2, 2], scope='pool3')

net = ...
net = slim.repeat(net, 3, slim,conv2d, 256, [3, 3], scope='conv3')
net = slim.max_pool(net, [2, 2], scope='pool3')

x = slim.fully_connected(x, 32, scope='fc/fc_1')
x = slim.fully_connected(x, 64, scope='fc/fc_2')
x = slim.fully_connected(x, 128, scope='fc/fc_3')

slim.stack(x, slim.fully_connected, [32, 64, 128], scope='fc')

TypeError: Expected binary or unicode string, got Ellipsis

# Argscope (p.61)

In [12]:
he_init = slim.variance_scaling_initializer() 
xavier_init = slim.xavier_initializer() 

with slim.arg_scope([slim.conv2d, slim.fully_connected], 
                    activation_fn=tf.nn.relu, 
                    weights_initializer=he_init, 
                    weights_regularizer=slim.l2_regularizer(0.0005)): 
    
    with slim.arg_scope([slim.conv2d], stride=1, padding='SAME'): 
        net = slim.conv2d(inputs, 64, [11, 11], 4, scope='conv1') 
        net = slim.conv2d(net, 256, [5, 5], weights_initializer=xavier_init, scope='conv2') 
        
        net = slim.fully_connected(net, 1000, activation_fn=None, scope='fc')

NameError: name 'inputs' is not defined

# Losses (p.64)

In [22]:
import tensorflow as tf

loss1 = slim.losses.softmax_cross_entropy(pred1, label1)
loss2 = slim.losses.mean_squared_error(pred2, label2)

total_loss = loss1 + loss2
slim.losses.get_total_loss(add_regulation_losses = False)

reg_loss = tf.add_n(slim.losses.get_regularization_losses())
total_loss = loss1 + loss2 + reg_loss

total_loss = slim.losses.get_total_loss()

NameError: name 'pred1' is not defined

# Collection (p.65)

In [23]:
def add_loss(loss, loss_collection=ops.GraphKeys.LOSSES):
    if loss_collection:
        ops.add_to_collection(loss_collection, loss)
        
def get_losses(scope=None, loss_collection=ops.GraphKeys.LOSSES):
    return ops.get_collection(loss_collection, scope)

def get_regularization_losses(scope=None):
    return ops.get_collection(ops.GraphKeys.REGULARIZATION_LOSSES, scope)

for var in tf.trainable_variables():
    print(var.name)
    
def trainable_variables():
    return ops.get_collection(ops.GraphKeys.TRAINABLE_VARIABLES)

NameError: name 'ops' is not defined

# Save/Restore (p.70)

In [24]:
import tensorflow as tf
def save(self, ckpt_dir, global_step=None): 
    if self.config.get("saver") is None: 
        self.config["saver"] = \
            tf.train.Saver(max_to_keep=30) 
            
    saver = self.config["saver"] 
    sess = self.config["sess"] 
            
    dirname = os.path.join(ckpt_dir, self.name) 
            
    if not os.path.exists(dirname): 
        os.makedirs(dirname) 
    saver.save(sess, dirname, global_step)

In [25]:
def load_latest_checkpoint(self, ckpt_dir, exclude=None):
    path = tf.train.latest_checkpoint(ckpt_dir)
    if path is None:
        raise AssertionError("No ckpt exists in {0}.".format(ckpt_dir))
        
    print("Load {} save file".format(path))
    self._load(path, exclude)
    
def load_from_path(self, ckpt_path, exclude=None):
    self._load(ckpt_path, exclude)
    
def _load(self, ckpt_path, exclude):
    init_fn = slim.assign_from_checkpoint_fn(ckpt_path, slim.get_variables_to_restore(exclude=exclude), ignore_missing_vars=True)
    init_fn(self.config["sess"])

# VGG-16 (p.76)

In [26]:
def vgg_16(inputs, num_classes=1000, is_training=True, dropout_keep_prob=0.5, spatial_squeeze=True, scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 2, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 2, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')            
            net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training, scope='dropout6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training, scope='dropout7')
            net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc8')        
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                end_points[sc.name + '/fc8'] = net
            return net, end_points
vgg_16.default_image_size = 224


def vgg_arg_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu, weights_regularizer=slim.l2_regularizer(weight_decay), biases_initializer=tf.zeros_initializer):
        with slim.arg_scope([slim.conv2d], padding='SAME') as arg_sc:
            return arg_sc

In [29]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with slim.arg_scope(my_arg_scope(is_training, weight_decay)):
    net, end_pts = my_net(x, keep_prob)
    pred = slim.softmax(net, scope="prediction")
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = cls_loss+reg_loss
    
with tf.variable_scope("Adam"):
    opt = tf.train.AdamOptimizer(lr)
    grads = tf.gradients(loss_op, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    
    apply_grads = opt.apply_gradients(grads_and_vars=grads)

with tf.variable_scope("accuracy"):
    correct_op = tf.equal(tf.argmax(net,1), tf.argmax(y,1))
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))

NameError: name 'my_arg_scope' is not defined

# Tensor Board & Slim example (p.82)

In [30]:
import tensorflow as tf
slim = tf.contrib.slim

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

max_steps = 10000
batch_size = 128
lr = 0.001
keep_prob = 0.5
weight_decay = 0.0004
logs_path = "/tmp/tensorflow_logs/example"

def my_arg_scope(is_training, weight_decay):
    with slim.arg_scope([slim.conv2d],
        activation_fn=tf.nn.relu,
        weights_regularizer=slim.l2_regularizer(weight_decay),
        weights_initializer=slim.variance_scaling_initializer(),
        biases_initializer=tf.zeros_initializer,
        stride=1, padding="SAME"):
        with slim.arg_scope([slim.dropout], is_training=is_training) as arg_sc:
            return arg_sc
        
def my_net(x, keep_prob, outputs_collections="my_net"):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    with slim.arg_scope([slim.conv2d, slim.max_pool2d],
        outputs_collections=outputs_collections):
        net = slim.conv2d(x, 64, [3, 3], scope="conv1")
        net = slim.max_pool2d(net, [2, 2], scope="pool1")
        net = slim.conv2d(net, 128, [3, 3], scope="conv2")
        net = slim.max_pool2d(net, [2, 2], scope="pool2")
        net = slim.conv2d(net, 256, [3, 3], scope="conv3")

        net = tf.reduce_mean(net, [1, 2], name="pool3", keep_dims=True)
        net = slim.dropout(net, keep_prob, scope="dropout3")
        net = slim.conv2d(net, 1024, [1, 1], scope="fc4")
        net = slim.dropout(net, keep_prob, scope="dropout4")
        net = slim.conv2d(net, 10, [1, 1], activation_fn=None, scope="fc5")
    end_points = \
        slim.utils.convert_collection_to_dict(outputs_collections)
    return tf.reshape(net, [-1, 10]), end_points

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with slim.arg_scope(my_arg_scope(is_training, weight_decay)):
    net, end_pts = my_net(x, keep_prob)
    pred = slim.softmax(net, scope="prediction")
    
with tf.variable_scope("losses"):
    cls_loss = slim.losses.softmax_cross_entropy(net, y)
    reg_loss = tf.add_n(slim.losses.get_regularization_losses())
    loss_op = cls_loss + reg_loss
    
with tf.variable_scope("Adam"):
    opt = tf.train.AdamOptimizer(lr)
    grads = tf.gradients(loss_op, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    apply_grads = opt.apply_gradients(grads_and_vars=grads)
    
with tf.variable_scope("accuracy"):
    correct_op = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_op, tf.float32))
    

summ_loss = tf.summary.scalar("loss", loss_op)
summ_acc = tf.summary.scalar("accuracy_test", acc_op)

for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var, collections=["my_summ"])
    
for grad, var in grads:
    tf.summary.histogram(var.name + "/gradient", grad,
collections=["my_summ"])
    
summ_wg = tf.summary.merge_all(key="my_summ")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)


for step in range(max_steps):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    _, loss, plot_loss, plot_wg = sess.run([apply_grads, loss_op, summ_loss, summ_wg],feed_dict={x: batch_X, y: batch_y, is_training: True})
    summary_writer.add_summary(plot_loss, step)
    summary_writer.add_summary(plot_wg, step)
    
    if (step+1) % 100 == 0:
        plot_acc = sess.run(summ_acc, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False})
        summary_writer.add_summary(plot_acc, step)
        
print("Optimization Finished!")
test_acc = sess.run(acc_op, feed_dict={x: mnist.test.images, y: mnist.test.labels, is_training: False})
print("Test accuracy: {:.3f}".format(test_acc))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


TypeError: __init__() got an unexpected keyword argument 'partition_info'